<a href="https://colab.research.google.com/github/hariomshahu/LLM/blob/main/llm_tool_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
from huggingface_hub import InferenceClient
from huggingface_hub import login
import json

In [10]:
from google.colab import userdata
groq_api_key = userdata.get('groq')

# Robust LLM JSON Output Validation and Prompt Engineering

In [13]:
import os
from huggingface_hub import InferenceClient
from pydantic import BaseModel, Field

class WeatherQuery(BaseModel):
    location: str = Field(description="City to check weather for")

client = InferenceClient(
    provider="groq",
    api_key=groq_api_key
)

messages = [
    {"role": "system", "content": "Respond only with a JSON object for the weather query."},
    {"role": "user", "content": "Weather in Paris?"}
]

response = client.chat.completions.create(
    model="meta-llama/Llama-4-Scout-17B-16E-Instruct",
    messages=messages
)
print(response.choices[0].message["content"])

weather_query = WeatherQuery.model_validate_json(response.choices[0].message["content"])
print(weather_query.location)


```json
{
  "city": "Paris",
  "country": "France",
  "temperature": 22,
  "conditions": "Partly Cloudy",
  "humidity": 60,
  "wind_speed": 15
}
```


ValidationError: 1 validation error for WeatherQuery
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='```json\n{\n  "city": "P...wind_speed": 15\n}\n```', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid

In [48]:
import os
import json
import time
from huggingface_hub import InferenceClient
from pydantic import BaseModel, Field, ValidationError

class WeatherQuery(BaseModel):
    location: str = Field(description="City to check weather for")

client = InferenceClient(
    provider="groq",
    api_key=groq_api_key
)

messages = [
    {"role": "system", "content": (
            "Respond only with a minified JSON object matching this schema: "
            '{"location": "<city name as string>"}. '
            "Do not include any Markdown formatting, code block markers, explanations, or extra text."
        )},
    {"role": "user", "content": "Weather in Paris?"}
]

MAX_RETRIES = 3

def query_with_retry(messages, max_retries=3, delay=2):
    last_exception = None
    for attempt in range(1, max_retries + 1):
        try:
            response = client.chat.completions.create(
                model="meta-llama/Llama-4-Scout-17B-16E-Instruct",
                messages=messages
            )
            output = response.choices[0].message["content"]
            # Attempt to parse and validate JSON output
            weather_query = WeatherQuery.model_validate_json(output)
            return weather_query
        except (json.JSONDecodeError, ValidationError) as e:
            print(f"Attempt {attempt}: Failed to parse/validate response. Error: {e}")
            last_exception = e
            if attempt < max_retries:
                time.sleep(delay)  # Wait before retrying
    raise RuntimeError(f"All {max_retries} attempts failed. Last error: {last_exception}")

try:
    weather_query = query_with_retry(messages, MAX_RETRIES)
    print(weather_query.location)
except RuntimeError as err:
    print(f"Error: {err}")


Paris


In [49]:
import os
import json
import time
from huggingface_hub import InferenceClient
from pydantic import BaseModel, Field, ValidationError

class WeatherQuery(BaseModel):
    location: str = Field(description="City to check weather for")

client = InferenceClient(
    provider="groq",
    api_key=groq_api_key
)

messages = [
    {"role": "system", "content": "Respond only with a JSON object for the weather query."},
    {"role": "user", "content": "Weather in Paris?"}
]

MAX_RETRIES = 3

def query_with_retry(messages, max_retries=3, delay=2):
    last_exception = None
    for attempt in range(1, max_retries + 1):
        try:
            response = client.chat.completions.create(
                model="meta-llama/Llama-4-Scout-17B-16E-Instruct",
                messages=messages
            )
            output = response.choices[0].message["content"]
            weather_query = WeatherQuery.model_validate_json(output)
            return weather_query
        except (json.JSONDecodeError, ValidationError) as e:
            print(f"Attempt {attempt}: Failed to parse/validate response. Error: {e}")
            last_exception = e
            if attempt < max_retries:
                time.sleep(delay)
    raise RuntimeError(f"All {max_retries} attempts failed. Last error: {last_exception}")

try:
    weather_query = query_with_retry(messages, MAX_RETRIES)
    print(weather_query.location)
except RuntimeError as err:
    print(f"Error: {err}")


Attempt 1: Failed to parse/validate response. Error: 1 validation error for WeatherQuery
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='```json\n{\n  "city": "P...wind_speed": 15\n}\n```', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid
Attempt 2: Failed to parse/validate response. Error: 1 validation error for WeatherQuery
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='```json\n{\n  "temperatu...wind_speed": 15\n}\n```', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid
Attempt 3: Failed to parse/validate response. Error: 1 validation error for WeatherQuery
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='```json\n{\n  "city": "P...wind_speed": 15\n}\n```', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid
Error: All 3 attempts failed. L

# Three-Stage Retry Mechanism When Using Tool Calls

In [55]:
client = InferenceClient(
    provider="groq",
    api_key=groq_api_key
)

In [56]:
import json
import inspect
import time
import random
from typing import Dict, Any, Callable, Optional, List
from dataclasses import dataclass
from enum import Enum

class RetryStage(Enum):
    """Three distinct retry stages"""
    LLM_API_CALL = "llm_api_call"
    TOOL_API_CALL = "tool_api_call"
    JSON_VALIDATION = "json_validation"

class RetryableError(Enum):
    """Types of errors that should trigger retries"""
    # Stage 1: LLM API Call errors
    LLM_NETWORK_ERROR = "llm_network_error"
    LLM_TIMEOUT_ERROR = "llm_timeout_error"
    LLM_RATE_LIMIT_ERROR = "llm_rate_limit_error"
    LLM_API_ERROR = "llm_api_error"

    # Stage 2: Tool API Call errors
    TOOL_NETWORK_ERROR = "tool_network_error"
    TOOL_TIMEOUT_ERROR = "tool_timeout_error"
    TOOL_RATE_LIMIT_ERROR = "tool_rate_limit_error"
    TOOL_API_ERROR = "tool_api_error"

    # Stage 3: JSON Validation errors
    MALFORMED_JSON = "malformed_json"
    INVALID_PARAMETERS = "invalid_parameters"
    MISSING_REQUIRED_PARAMS = "missing_required_params"
    TYPE_MISMATCH = "type_mismatch"

@dataclass
class StageRetryConfig:
    """Configuration for each retry stage"""
    max_retries: int = 3
    base_delay: float = 1.0
    max_delay: float = 60.0
    backoff_multiplier: float = 2.0
    jitter: bool = True
    retryable_errors: List[RetryableError] = None
    temperature_adjustment: float = 0.0  # For LLM retries

    def __post_init__(self):
        if self.retryable_errors is None:
            self.retryable_errors = []

@dataclass
class ThreeStageRetryConfig:
    """Configuration for all three retry stages"""
    # Stage 1: LLM API Call retries
    llm_api_retry: StageRetryConfig = None

    # Stage 2: Tool API Call retries
    tool_api_retry: StageRetryConfig = None

    # Stage 3: JSON Validation retries
    json_validation_retry: StageRetryConfig = None

    def __post_init__(self):
        if self.llm_api_retry is None:
            self.llm_api_retry = StageRetryConfig(
                max_retries=3,
                base_delay=1.0,
                backoff_multiplier=2.0,
                temperature_adjustment=0.1,
                retryable_errors=[
                    RetryableError.LLM_NETWORK_ERROR,
                    RetryableError.LLM_TIMEOUT_ERROR,
                    RetryableError.LLM_RATE_LIMIT_ERROR,
                    RetryableError.LLM_API_ERROR
                ]
            )

        if self.tool_api_retry is None:
            self.tool_api_retry = StageRetryConfig(
                max_retries=3,
                base_delay=0.5,
                backoff_multiplier=1.5,
                retryable_errors=[
                    RetryableError.TOOL_NETWORK_ERROR,
                    RetryableError.TOOL_TIMEOUT_ERROR,
                    RetryableError.TOOL_RATE_LIMIT_ERROR,
                    RetryableError.TOOL_API_ERROR
                ]
            )

        if self.json_validation_retry is None:
            self.json_validation_retry = StageRetryConfig(
                max_retries=2,
                base_delay=0.3,
                backoff_multiplier=1.2,
                temperature_adjustment=0.15,
                retryable_errors=[
                    RetryableError.MALFORMED_JSON,
                    RetryableError.INVALID_PARAMETERS,
                    RetryableError.MISSING_REQUIRED_PARAMS,
                    RetryableError.TYPE_MISMATCH
                ]
            )

def calculate_delay(attempt: int, config: StageRetryConfig) -> float:
    delay = config.base_delay * (config.backoff_multiplier ** attempt)
    delay = min(delay, config.max_delay)

    if config.jitter:
        jitter_range = delay * 0.25
        delay += random.uniform(-jitter_range, jitter_range)

    return max(0, delay)

def classify_llm_api_error(error: Exception, error_msg: str) -> Optional[RetryableError]:
    error_msg_lower = error_msg.lower()

    if any(keyword in error_msg_lower for keyword in ['connection', 'network', 'dns', 'socket']):
        return RetryableError.LLM_NETWORK_ERROR
    if any(keyword in error_msg_lower for keyword in ['timeout', 'timed out']):
        return RetryableError.LLM_TIMEOUT_ERROR
    if any(keyword in error_msg_lower for keyword in ['rate limit', 'too many requests', '429']):
        return RetryableError.LLM_RATE_LIMIT_ERROR
    if any(keyword in error_msg_lower for keyword in ['api', 'service', '503', '500']):
        return RetryableError.LLM_API_ERROR

    return None

def classify_tool_api_error(error: Exception, error_msg: str) -> Optional[RetryableError]:
    error_msg_lower = error_msg.lower()

    if any(keyword in error_msg_lower for keyword in ['connection', 'network', 'dns', 'socket']):
        return RetryableError.TOOL_NETWORK_ERROR
    if any(keyword in error_msg_lower for keyword in ['timeout', 'timed out']):
        return RetryableError.TOOL_TIMEOUT_ERROR
    if any(keyword in error_msg_lower for keyword in ['rate limit', 'too many requests', '429']):
        return RetryableError.TOOL_RATE_LIMIT_ERROR
    if any(keyword in error_msg_lower for keyword in ['api', 'service', '503', '500']):
        return RetryableError.TOOL_API_ERROR

    return None

def classify_json_validation_error(error: Exception, error_msg: str) -> Optional[RetryableError]:
    error_msg_lower = error_msg.lower()

    if any(keyword in error_msg_lower for keyword in ['json', 'decode', 'parse']):
        return RetryableError.MALFORMED_JSON
    if any(keyword in error_msg_lower for keyword in ['missing', 'required']):
        return RetryableError.MISSING_REQUIRED_PARAMS
    if any(keyword in error_msg_lower for keyword in ['invalid', 'unexpected']):
        return RetryableError.INVALID_PARAMETERS
    if any(keyword in error_msg_lower for keyword in ['type', 'mismatch', 'wrong type']):
        return RetryableError.TYPE_MISMATCH

    return None

# weather tools (with simulated failures for testing)
def get_temperature(location: str):
    """Get the temperature for a given location"""
    # Simulate occasional tool API failures (Stage 2)
    if random.random() < 0.15:
        raise ConnectionError("Weather API connection error")

    temperatures = {"New York": "22°C", "London": "18°C", "Tokyo": "26°C", "Sydney": "20°C"}
    return temperatures.get(location, "Temperature data not available")

def get_weather_condition(location: str):
    """Get the weather condition for a given location"""
    # Simulate occasional tool API failures (Stage 2)
    if random.random() < 0.15:
        raise TimeoutError("Weather API timeout")

    conditions = {"New York": "Sunny", "London": "Rainy", "Tokyo": "Cloudy", "Sydney": "Clear"}
    return conditions.get(location, "Weather condition data not available")

def stage1_llm_api_call_with_retry(client, model, messages, tools, tool_choice, max_tokens, temperature, retry_config):
    """
    Stage 1: LLM API Call with retry mechanism
    Handles LLM service errors, network issues, rate limits
    """
    print(f"\n STAGE 1: LLM API Call")

    last_error = None
    retry_history = []
    current_temperature = temperature

    for attempt in range(retry_config.max_retries + 1):
        try:
            print(f"  Attempt {attempt + 1}/{retry_config.max_retries + 1} (temp: {current_temperature:.1f})")

            response = client.chat.completions.create(
                model=model,
                messages=messages,
                tools=tools,
                tool_choice=tool_choice,
                max_tokens=max_tokens,
                temperature=current_temperature
            )

            if retry_history:
                print(f"   LLM API succeeded after {attempt} retries")
            else:
                print(f"   LLM API succeeded on first attempt")

            return {"success": True, "response": response, "retry_history": retry_history}

        except Exception as e:
            last_error = e
            error_msg = str(e)

            retryable_error_type = classify_llm_api_error(e, error_msg)
            is_retryable = (
                retryable_error_type is not None and
                retryable_error_type in retry_config.retryable_errors and
                attempt < retry_config.max_retries
            )

            retry_info = {
                "stage": "llm_api_call",
                "attempt": attempt + 1,
                "error": error_msg,
                "error_type": type(e).__name__,
                "retryable_error_type": retryable_error_type.value if retryable_error_type else None,
                "timestamp": time.time()
            }
            retry_history.append(retry_info)

            if is_retryable:
                delay = calculate_delay(attempt, retry_config)
                print(f"   LLM API failed: {error_msg}")
                print(f"   Retrying in {delay:.2f}s...")

                retry_info["delay"] = delay
                time.sleep(delay)

                # Adjust temperature for different responses
                current_temperature = min(current_temperature + retry_config.temperature_adjustment, 0.9)
            else:
                print(f"   LLM API failed (non-retryable): {error_msg}")
                break

    return {
        "success": False,
        "error": f"LLM API failed: {str(last_error)}",
        "retry_history": retry_history
    }

def stage3_json_validation_with_retry(tool_call, available_functions, retry_config, client, model, messages, tools, tool_choice, max_tokens, base_temperature):
    """
    Stage 3: JSON Validation with retry mechanism
    Handles malformed JSON, wrong parameters, type mismatches
    """
    print(f"\n STAGE 3: JSON Validation for {tool_call.function.name}")

    last_error = None
    retry_history = []
    current_temperature = base_temperature
    current_tool_call = tool_call

    for attempt in range(retry_config.max_retries + 1):
        try:
            print(f"  Attempt {attempt + 1}/{retry_config.max_retries + 1}")

            function_name = current_tool_call.function.name

            if function_name not in available_functions:
                raise ValueError(f"Function '{function_name}' not found in available functions")

            function_to_call = available_functions[function_name]

            try:
                function_args = json.loads(current_tool_call.function.arguments)
            except json.JSONDecodeError as e:
                raise ValueError(f"Malformed JSON in function arguments: {str(e)}")

            sig = inspect.signature(function_to_call)
            expected_params = list(sig.parameters.keys())

            required_params = [
                name for name, param in sig.parameters.items()
                if param.default == inspect.Parameter.empty
            ]

            missing_params = [param for param in required_params if param not in function_args]
            if missing_params:
                raise ValueError(f"Missing required parameters: {missing_params}")

            unexpected_params = [k for k in function_args.keys() if k not in expected_params]
            if unexpected_params:
                raise ValueError(f"Invalid parameters: {unexpected_params}")

            valid_args = {k: v for k, v in function_args.items() if k in expected_params}

            if retry_history:
                print(f"   JSON validation succeeded after {attempt} retries")
            else:
                print(f"   JSON validation succeeded on first attempt")

            return {
                "success": True,
                "function_to_call": function_to_call,
                "valid_args": valid_args,
                "retry_history": retry_history
            }

        except Exception as e:
            last_error = e
            error_msg = str(e)

            retryable_error_type = classify_json_validation_error(e, error_msg)
            is_retryable = (
                retryable_error_type is not None and
                retryable_error_type in retry_config.retryable_errors and
                attempt < retry_config.max_retries
            )

            retry_info = {
                "stage": "json_validation",
                "attempt": attempt + 1,
                "error": error_msg,
                "error_type": type(e).__name__,
                "retryable_error_type": retryable_error_type.value if retryable_error_type else None,
                "timestamp": time.time()
            }
            retry_history.append(retry_info)

            if is_retryable:
                delay = calculate_delay(attempt, retry_config)
                print(f"   JSON validation failed: {error_msg}")
                print(f"   Re-calling LLM in {delay:.2f}s...")

                retry_info["delay"] = delay
                time.sleep(delay)

                # Increase temperature and retry LLM call to get different JSON
                current_temperature = min(current_temperature + retry_config.temperature_adjustment, 0.9)

                # Make new LLM call to get corrected JSON
                llm_result = stage1_llm_api_call_with_retry(
                    client, model, messages[:1] + [{"role": "user", "content": f"Please provide the correct tool call for {function_name}. The previous attempt had this error: {error_msg}"}],
                    tools, tool_choice, max_tokens, current_temperature,
                    StageRetryConfig(max_retries=1, base_delay=0.5)
                )

                if llm_result["success"]:
                    new_response = llm_result["response"]
                    if hasattr(new_response.choices[0].message, 'tool_calls') and new_response.choices[0].message.tool_calls:
                        current_tool_call = new_response.choices[0].message.tool_calls[0]
                        print(f"   Got new tool call, retrying validation...")
                    else:
                        print(f"   LLM didn't provide tool calls in retry")
                        break
                else:
                    print(f"   Failed to get new LLM response")
                    break
            else:
                print(f"   JSON validation failed (non-retryable): {error_msg}")
                break

    return {
        "success": False,
        "error": f"JSON validation failed: {str(last_error)}",
        "retry_history": retry_history
    }

def stage2_tool_api_call_with_retry(function_to_call, function_args, function_name, retry_config):
    """
    Stage 2: Tool API Call with retry mechanism
    Handles tool execution errors, network issues, API failures
    """
    print(f"\n STAGE 2: Tool API Call for {function_name}")

    last_error = None
    retry_history = []

    for attempt in range(retry_config.max_retries + 1):
        try:
            print(f"  Attempt {attempt + 1}/{retry_config.max_retries + 1}")

            result = function_to_call(**function_args)

            if retry_history:
                print(f"   Tool API succeeded after {attempt} retries")
            else:
                print(f"   Tool API succeeded on first attempt")

            return {"success": True, "result": result, "retry_history": retry_history}

        except Exception as e:
            last_error = e
            error_msg = str(e)

            retryable_error_type = classify_tool_api_error(e, error_msg)
            is_retryable = (
                retryable_error_type is not None and
                retryable_error_type in retry_config.retryable_errors and
                attempt < retry_config.max_retries
            )

            retry_info = {
                "stage": "tool_api_call",
                "attempt": attempt + 1,
                "error": error_msg,
                "error_type": type(e).__name__,
                "retryable_error_type": retryable_error_type.value if retryable_error_type else None,
                "timestamp": time.time()
            }
            retry_history.append(retry_info)

            if is_retryable:
                delay = calculate_delay(attempt, retry_config)
                print(f"   Tool API failed: {error_msg}")
                print(f"   Retrying in {delay:.2f}s...")

                retry_info["delay"] = delay
                time.sleep(delay)
            else:
                print(f"   Tool API failed (non-retryable): {error_msg}")
                break

    return {
        "success": False,
        "error": f"Tool API failed: {str(last_error)}",
        "retry_history": retry_history
    }

def three_stage_tool_execution(tool_call, available_functions, retry_config, client, model, messages, tools, tool_choice, max_tokens, base_temperature):

    print(f"\n Processing tool call: {tool_call.function.name}")

    all_retry_history = []

    # Stage 3: JSON Validation
    validation_result = stage3_json_validation_with_retry(
        tool_call, available_functions, retry_config.json_validation_retry,
        client, model, messages, tools, tool_choice, max_tokens, base_temperature
    )

    if validation_result["retry_history"]:
        all_retry_history.extend(validation_result["retry_history"])

    if not validation_result["success"]:
        return {
            "success": False,
            "error": validation_result["error"],
            "retry_history": all_retry_history,
            "failed_stage": "json_validation"
        }

    # Stage 2: Tool API Call
    execution_result = stage2_tool_api_call_with_retry(
        validation_result["function_to_call"],
        validation_result["valid_args"],
        tool_call.function.name,
        retry_config.tool_api_retry
    )

    if execution_result["retry_history"]:
        all_retry_history.extend(execution_result["retry_history"])

    if not execution_result["success"]:
        return {
            "success": False,
            "error": execution_result["error"],
            "retry_history": all_retry_history,
            "failed_stage": "tool_api_call"
        }

    return {
        "success": True,
        "result": execution_result["result"],
        "retry_history": all_retry_history
    }

if __name__ == "__main__":
    # Initialize retry configuration
    retry_config = ThreeStageRetryConfig()

    messages = [
        {"role": "system", "content": "You are a helpful weather assistant."},
        {"role": "user", "content": "What's the weather and temperature like in New York and London? Respond with one sentence for each city. Use tools to get the information."},
    ]

    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_temperature",
                "description": "Get the temperature for a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The name of the city",
                        }
                    },
                    "required": ["location"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_weather_condition",
                "description": "Get the weather condition for a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The name of the city",
                        }
                    },
                    "required": ["location"],
                },
            },
        }
    ]

    available_functions = {
        "get_temperature": get_temperature,
        "get_weather_condition": get_weather_condition,
    }

    # Stage 1: Initial LLM API call
    llm_result = stage1_llm_api_call_with_retry(
        client=client,
        model="meta-llama/Llama-4-Scout-17B-16E-Instruct",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096,
        temperature=0.5,
        retry_config=retry_config.llm_api_retry
    )

    if not llm_result["success"]:
        print(f" FAILED: {llm_result['error']}")
        exit(1)

    response = llm_result["response"]
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    # Process tool calls
    messages.append(response_message)

    for tool_call in tool_calls:
        execution_result = three_stage_tool_execution(
            tool_call, available_functions, retry_config,
            client, "meta-llama/Llama-4-Scout-17B-16E-Instruct",
            messages, tools, "auto", 4096, 0.5
        )

        if execution_result["success"]:
            function_response = execution_result["result"]
            retry_summary = ""
            if execution_result["retry_history"]:
                total_retries = len(execution_result["retry_history"])
                retry_summary = f" (succeeded after {total_retries} total retries across stages)"
            print(f" Overall success for {tool_call.function.name}{retry_summary}: {function_response}")
        else:
            error_msg = execution_result["error"]
            failed_stage = execution_result.get("failed_stage", "unknown")
            retry_summary = ""
            if execution_result["retry_history"]:
                total_retries = len(execution_result["retry_history"])
                retry_summary = f" (failed after {total_retries} total retries, failed at {failed_stage} stage)"
            print(f" Overall failure for {tool_call.function.name}{retry_summary}: {error_msg}")
            function_response = f"Error: {error_msg}"

        # Add tool response to messages
        messages.append({
            "role": "tool",
            "content": str(function_response),
            "tool_call_id": tool_call.id,
        })

    # Stage 1: Final LLM API call
    print(f"\n FINAL LLM CALL")
    final_llm_result = stage1_llm_api_call_with_retry(
        client=client,
        model="meta-llama/Llama-4-Scout-17B-16E-Instruct",
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096,
        temperature=0.5,
        retry_config=retry_config.llm_api_retry
    )

    if final_llm_result["success"]:
        final_response = final_llm_result["response"]
        print(f"\n FINAL RESPONSE:")
        print(final_response.choices[0].message.content)
    else:
        print(f" FINAL LLM CALL FAILED: {final_llm_result['error']}")


 STAGE 1: LLM API Call
  Attempt 1/4 (temp: 0.5)
   LLM API succeeded on first attempt

 Processing tool call: get_weather_condition

 STAGE 3: JSON Validation for get_weather_condition
  Attempt 1/3
   JSON validation succeeded on first attempt

 STAGE 2: Tool API Call for get_weather_condition
  Attempt 1/4
   Tool API succeeded on first attempt
 Overall success for get_weather_condition: Sunny

 Processing tool call: get_temperature

 STAGE 3: JSON Validation for get_temperature
  Attempt 1/3
   JSON validation succeeded on first attempt

 STAGE 2: Tool API Call for get_temperature
  Attempt 1/4
   Tool API succeeded on first attempt
 Overall success for get_temperature: 22°C

 FINAL LLM CALL

 STAGE 1: LLM API Call
  Attempt 1/4 (temp: 0.5)
   LLM API succeeded on first attempt

 FINAL RESPONSE:
The weather in New York is sunny with a temperature of 22°C.


